In [1]:
import pandas as pd
import gspread
from google.auth import default
from google.colab import drive
from google.colab import auth
drive.mount('/content/gdrive')
creds, _ = default()
gc = gspread.authorize(creds)
auth.authenticate_user()

Mounted at /content/gdrive


In [2]:
FIRST_TEAM_NUM = 1101 #Team ID of first team
LAST_TEAM_NUM = 1477 #Team ID of last team
FIRST_TEAM_NUM_W = 3101 #Team ID of first team (women's)
LAST_TEAM_NUM_W = 3477 #Team ID of last team (women's)
NUM_TEAMS = LAST_TEAM_NUM - FIRST_TEAM_NUM + 1
PATH = "/content/gdrive/MyDrive/ZSchool_Projects/Purdue/Fun/Kaggle2023/" #Path to storage of Kaggle provided data

In [35]:
def getData(w=False):
  if w:
    #worksheet = gc.open('WRegularSeasonDetailedResults').sheet1
    worksheet = gc.open('2023TrainW').sheet1
    rows = worksheet.get_all_values()
    data = pd.DataFrame.from_records(rows)
    data2 = pd.read_csv(PATH + "WNCAATourneySeedsReal.csv")
  else:
    #worksheet = gc.open('MRegularSeasonDetailedResults').sheet1
    worksheet = gc.open('2023TrainM').sheet1
    rows = worksheet.get_all_values()
    data = pd.DataFrame.from_records(rows)
    data2 = pd.read_csv(PATH + "MNCAATourneySeedsReal.csv")

  data.columns=data.iloc[0]
  data = data[1:]

  return data, data2

In [36]:
def writeData(colName, year, data, w=False):
  totals = []
  teamList = []
  teamCountList = [0] * len(data[colName]) #number of times each time comes up in data so you can divide by this to get average for the season

  for index in range(1, 1 + len(data[colName])):
    if data["Season"][index] == str(year):
      if int((data["WTeamID"][index])) in teamList:
        trueIndex = teamList.index(int(data["WTeamID"][index]))
        totals[trueIndex] += float(data[colName][index]) + float(data["WDR"][index])
        teamCountList[trueIndex] += 1
      else:
        totals.append(float(data[colName][index]) + float(data["WDR"][index]))
        teamList.append(int(data["WTeamID"][index]))
        teamCountList[index] += 1

  for index in range(len(totals)):
    if teamCountList[index] != 0:
      totals[index] /= teamCountList[index]
    else:
      teamCountList[index] == 1
      totals[index] == -100

  totals = pd.DataFrame(totals)
  
  n_rebounds = totals.values.tolist() #((totals - totals.mean()) / totals.std()).values.tolist()

  if w:
    woman = "w"
  else:
    woman = ""

  diffList = getDiff(year, teamList, n_rebounds, data)

  with open(PATH + colName + woman + "boogaloo.csv", "a") as file:
    for index in range(len(diffList)): #teamList)):
      #file.write(f"{teamList[index]},{n_rebounds[index][0]},{year}\n")
      file.write(f"{diffList[index]}\n")

In [32]:
def writeDataTemp(colName, year, data, data2, w=False):
  totals = []
  teamList = []

  print(year, colName)

  for index in range(1, len(data2[colName])):
    if data2["Season"][index] == year:
      totals.append(float(data2[colName][index][1:3]))
      teamList.append(int(data2["TeamID"][index]))

  totals = pd.DataFrame(totals)
  
  n_rebounds = totals.values.tolist() #((totals - totals.mean()) / totals.std()).values.tolist()

  if w:
    woman = "w"
  else:
    woman = ""

  print("n_rebounds", n_rebounds)  
  
  diffList = getDiff(year, teamList, n_rebounds, data)

  print(diffList)

  with open(PATH + colName + woman + ".csv", "w") as file:
    for index in range(len(diffList)): #teamList)):
      #file.write(f"{teamList[index]},{n_rebounds[index][0]},{year}\n")
      file.write(f"{diffList[index]}\n")

In [33]:
def getDiff(year, teamList, n_rebounds, data):

  diffList = []

  for index in range(1, 1 + len(data["Season"])):
    if data["Season"][index] == str(year):
      try:
        diffList.append(n_rebounds[teamList.index(int(data["Team2"][index]))][0] - n_rebounds[teamList.index(int(data["Team1"][index]))][0])
      except Exception as e:
        print(e)
        diffList.append(-99999)

  return diffList




In [38]:
data, data2 = getData(w=True)

for year in [2023]: #[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]:
  writeDataTemp("Seed", year, data, data2, w=True)
  print(year)

Streaming output truncated to the last 5000 lines.
3332 is not in list
3333 is not in list
3415 is not in list
3338 is not in list
3396 is not in list
3344 is not in list
3350 is not in list
3384 is not in list
3353 is not in list
3216 is not in list
3365 is not in list
3389 is not in list
3427 is not in list
3374 is not in list
3190 is not in list
3287 is not in list
3307 is not in list
3386 is not in list
3388 is not in list
3361 is not in list
3393 is not in list
3395 is not in list
3123 is not in list
3403 is not in list
3460 is not in list
3407 is not in list
3408 is not in list
3409 is not in list
3457 is not in list
3142 is not in list
3418 is not in list
3337 is not in list
3426 is not in list
3225 is not in list
3431 is not in list
3433 is not in list
3434 is not in list
3443 is not in list
3265 is not in list
3438 is not in list
3291 is not in list
3447 is not in list
3448 is not in list
3469 is not in list
3258 is not in list
3456 is not in list
3458 is not in list
3462 is n

In [7]:
#Getting 2023 data

def getData2(w=False):
  if w:
    worksheet = gc.open('2023TrainW').sheet1
    rows = worksheet.get_all_values()
    data = pd.DataFrame.from_records(rows)
  else:
    worksheet = gc.open('2023TrainM').sheet1
    rows = worksheet.get_all_values()
    data = pd.DataFrame.from_records(rows)

  data.columns=data.iloc[0]
  data = data[1:]

  return data

In [8]:
def getData2023():
  teamList = []
  reboundList = []
  turnoverList = []
  teamCountList = [0] * len(data["Team1"]) #number of times each time comes up in data so you can divide by this to get average for the season

  for index in range(1, 1 + len(data["Team1"])):
    if data["Season"][index] == str(2023):
      if int((data["WTeamID"][index])) in teamList:
        trueIndex = teamList.index(int(data["WTeamID"][index]))
        reboundList[trueIndex] += float(data["Rebound"][index])
        turnoverList[trueIndex] += float(data["Turnover"][index])
        teamCountList[trueIndex] += 1
      else:
        teamList.append(int(data["WTeamID"][index]))
        teamCountList[index] += 1

  for index in range(len(totals)):
    if teamCountList[index] != 0:
      totals[index] /= teamCountList[index]
    else:
      teamCountList[index] == 1
      totals[index] == -100


In [ ]:
data = getData2()



In [ ]:
#------------------3/14 getting 2023 data setup

In [26]:
w=True

if w:
  worksheet = gc.open('2023TrainW').sheet1
  rows = worksheet.get_all_values()
  data = pd.DataFrame.from_records(rows)

  worksheet2 = gc.open('2023TrainW').get_worksheet(1)
  rows2 = worksheet2.get_all_values()
  data2 = pd.DataFrame.from_records(rows2)
else:
  worksheet = gc.open('2023TrainM').sheet1
  rows = worksheet.get_all_values()
  data = pd.DataFrame.from_records(rows)

  worksheet2 = gc.open('2023TrainM').get_worksheet(1)
  rows2 = worksheet2.get_all_values()
  data2 = pd.DataFrame.from_records(rows2)

data.columns=data.iloc[0]
data = data[1:]
data2.columns=data2.iloc[0]
data2 = data2[1:]


avgList = []

for index1 in range(1, 1+len(data2["TeamID"])):
  thisTeam = int(data2["TeamID"][index1])
  sum = 0
  count = 0
  for index2 in range(1, 1+len(data["Team1"])):
    if int(data["WTeamID"][index2]) == thisTeam:
      sum += float(data["WTO"][index2])
      count += 1
    
  avgList.append(sum / count)


with open(PATH + "WORWDR2023.csv", "w") as file:
    for index in range(len(avgList)): #teamList)):
      file.write(f"{avgList[index]}\n")
